#Credit Card Fraud Detection

##Importing the libraries

In [35]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

##Loading the Dataset

In [36]:
# Load the dataset
file_path = 'fraud_data.csv'
data = pd.read_csv(file_path)

# Step 1: Separate X and y
y = data['is_fraud']
X = data.drop(columns=['is_fraud', 'trans_num'])

##Data Preprocessing

In [37]:
# Clean the y column
y_cleaned = y.str.extract(r'(\d)').astype(int)
y = y_cleaned[0]

# Step 2: Convert 'trans_date_trans_time' and 'dob' to datetime
X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time'], format='%d-%m-%Y %H:%M', errors='coerce')
X['dob'] = pd.to_datetime(X['dob'], format='%d-%m-%Y', errors='coerce')

# Step 3: One-hot encoding for categorical variables
X = pd.get_dummies(X, columns=['merchant', 'category', 'city', 'job', 'state'], drop_first=True)

# Step 4: Feature Engineering
X['trans_date'] = X['trans_date_trans_time'].dt.date.apply(lambda x: x.toordinal())
X['trans_time'] = X['trans_date_trans_time'].dt.time.apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

X['age'] = (pd.to_datetime('today') - X['dob']).dt.days // 365

X.drop(columns=['trans_date_trans_time', 'dob'], inplace=True)

X['distance'] = X.apply(lambda row: geodesic((row['lat'], row['long']), (row['merch_lat'], row['merch_long'])).km, axis=1)
X.drop(columns=['lat', 'long', 'merch_lat', 'merch_long'], inplace=True)

# Step 5: Scaling/Normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


##Splitting the Data into Training and Test Sets

In [38]:
# Step 6: Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Building and Training the Artificial Neural Network

In [39]:
# Step 7: Build the ANN
ann = tf.keras.models.Sequential()

# Input layer
ann.add(tf.keras.layers.Dense(units=12, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden layers
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))

# Output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = ann.fit(X_train, y_train, batch_size=500, epochs=50, validation_data=(X_test, y_test))


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5338 - loss: 0.7306 - val_accuracy: 0.7450 - val_loss: 0.6379
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8083 - loss: 0.6151 - val_accuracy: 0.8412 - val_loss: 0.5692
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8783 - loss: 0.5438 - val_accuracy: 0.8657 - val_loss: 0.5051
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8943 - loss: 0.4729 - val_accuracy: 0.8758 - val_loss: 0.4483
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9085 - loss: 0.4066 - val_accuracy: 0.8848 - val_loss: 0.4004
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9136 - loss: 0.3552 - val_accuracy: 0.8948 - val_loss: 0.3607
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9196 - loss: 0.3112 - val_accuracy: 0.9031 - val_loss: 0.3281
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9254 - loss: 0.2755 - val_accuracy: 0.9066 - val_loss: 0.3035

##Model Evaluation

In [40]:
# Evaluate the model on the test set
loss, accuracy = ann.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Predict on the test set
y_pred = (ann.predict(X_test) > 0.5).astype("int32")

# Display classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9393 - loss: 0.2906
Test Accuracy: 0.9298
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      2505
           1       0.83      0.59      0.69       385

    accuracy                           0.93      2890
   macro avg       0.89      0.79      0.83      2890
weighted avg       0.93      0.93      0.92      2890

[[2459   46]
 [ 157  228]]
